In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Context

In [22]:
from dataclasses import dataclass

@dataclass
class BankContext:
    balance: int = 900
    last_withdrawal: int = 100
    last_deposit: int = 50

In [23]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    context_schema=BankContext  
)

In [14]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my bank balance?")]},
    context=BankContext()
)

In [15]:
print(response["messages"][-1].content)

I don’t have access to your bank accounts, so I can’t see or tell you your balance.

Here are quick ways to check securely:
- Mobile banking app: open the app, sign in, and look under Accounts or Balances.
- Online banking (website): sign in and view your Accounts/Balances page.
- ATM: insert your card and choose Balance or Account Summary.
- Phone banking: call your bank’s automated service and follow the prompts.
- In person: visit a branch with ID.

Tips:
- Enable balance alerts so you get notified of changes.
- Review recent transactions for any unfamiliar activity.

If you’d like, I can help you with budgeting or planning once you share non-sensitive details (e.g., your current balance and monthly expenses you want to track).


In [16]:
from langchain.tools import tool, ToolRuntime

@tool
def get_balance(runtime: ToolRuntime) -> str:
    """Get the bank balance of the user"""
    return runtime.context.balance

@tool
def get_last_withdrawal(runtime: ToolRuntime) -> str:
    """Get the last bank withdrawal of the user"""
    return runtime.context.last_withdrawal

@tool
def get_last_deposit(runtime: ToolRuntime) -> str:
    """Get the last bank deposit of the user"""
    return runtime.context.last_deposit   

In [17]:
agent = create_agent(
    model="gpt-5-nano",
    tools=[get_balance, get_last_deposit, get_last_withdrawal],
    context_schema=BankContext
)

In [18]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my bank balance?")]},
    context=BankContext()
)

/Users/suvo/Documents/Langchain/lca-lc-foundations/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=BankContext(balance=900, last_withdrawal=100), input_type=BankContext])
  return self.__pydantic_serializer__.to_python(


In [19]:
print(response["messages"][-1].content)

Your current balance is 900.

Would you like me to show your last withdrawal or recent transactions?


In [20]:
from langchain.messages import HumanMessage

response = agent.invoke(
    {"messages": [HumanMessage(content="What is my lat withdrawal?")]},
    context=BankContext()
)

/Users/suvo/Documents/Langchain/lca-lc-foundations/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=BankContext(balance=900, last_withdrawal=100), input_type=BankContext])
  return self.__pydantic_serializer__.to_python(


In [21]:
print(response["messages"][-1].content)

Your last withdrawal amount is 100.

Would you like me to fetch the date and location of that withdrawal or show your current balance as well?


## State

In [87]:
from langchain.agents import AgentState

class CustomState(AgentState):
    balance: int
    last_withdrawal: int
    last_deposit: int

In [88]:
@dataclass
class BankContext:
    balance: int = 900
    last_withdrawal: int = 100
    last_deposit: int = 0

In [100]:
from langgraph.types import Command
from langchain_core.messages import ToolMessage

@tool
def get_balance(runtime: ToolRuntime) -> str:
    """Get the bank balance of the user"""
    return runtime.context.balance

@tool
def get_last_withdrawal(runtime: ToolRuntime) -> str:
    """Get the last bank withdrawal of the user"""
    return runtime.context.last_withdrawal

@tool
def get_last_deposit(runtime: ToolRuntime) -> str:
    """Get the last bank deposit of the user"""
    return runtime.context.last_deposit 


@tool
def update_deposit(deposit: int, runtime: ToolRuntime) -> Command:
    """Update the deposit once user have entered the amount."""
    return Command(update={
        "last_deposit": deposit, 
        "messages": [ToolMessage("Successfully updated last deposit", tool_call_id=runtime.tool_call_id)]}
        )

@tool
def update_withdrawal(withdrawal: int, runtime: ToolRuntime) -> Command:
    """Update the withdrawal once user have entered the amount."""
    return Command(update={
        "last_withdrawal": withdrawal, 
        "messages": [ToolMessage("Successfully updated last withdrawal", tool_call_id=runtime.tool_call_id)]}
        )

@tool
def update_balance(deposit: int, withdrawal: int, runtime: ToolRuntime) -> Command:
    """Update the balance once a deposit/withdrawal is made."""
    current = runtime.state.get("balance", runtime.context.balance)

    deposit = abs(int(deposit))
    withdrawal = abs(int(withdrawal))

    new_balance = current + deposit - withdrawal
    return Command(update={
        "balance": new_balance, 
        "messages": [ToolMessage("Successfully updated balance", tool_call_id=runtime.tool_call_id)]}
        )

@tool
def read_balance(runtime: ToolRuntime) -> str:
    """Read the bank balance."""
    try:
        return runtime.state.get("balance", runtime.context.balance)
    except KeyError:
        return "No bank information found"

  

In [101]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    "gpt-5-nano",
    tools=[get_balance, get_last_deposit, get_last_withdrawal, update_deposit, update_withdrawal, update_balance, read_balance],
    checkpointer=InMemorySaver(),
    state_schema=CustomState,
    context_schema=BankContext
)

In [102]:
response = agent.invoke(
    { "messages": [HumanMessage(content="I did not add any deposit or make any withdrawal. What is my balance?")]},
    {"configurable": {"thread_id": "10"}},
    context=BankContext()
)

/Users/suvo/Documents/Langchain/lca-lc-foundations/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=BankContext(balance=900, ...wal=100, last_deposit=0), input_type=BankContext])
  return self.__pydantic_serializer__.to_python(


In [103]:
print(response["messages"][-1].content)

Your current balance is 900.

Would you like me to show your last deposit and last withdrawal details as well?


In [106]:
response = agent.invoke(
    {"messages": [HumanMessage(content="I made a deposit of 200 and a withdrawal of 100. What is my balance after the deposit and withdrawal?")]},
    {"configurable": {"thread_id": "15"}},
    context=BankContext()
)

In [107]:
print(response["messages"][-1].content)

Math says your balance would be 1000 now: 900 + 200 (deposit) - 100 (withdrawal) = 1000.

However, the current system balance still shows 900. If you want, I can update the balance to reflect these changes (set it to 1000) by applying the 200 deposit and 100 withdrawal. Shall I proceed to update the balance to 1000?


In [108]:
response = agent.invoke(
  {"messages": [
      ("system", "If the user states a deposit and/or withdrawal, you MUST call update_balance with those amounts, then call read_balance and answer with the stored balance. Do not ask for confirmation."),
      HumanMessage(content="I made a deposit of 200 and a withdrawal of 100. What is my balance?")
  ]},
  {"configurable": {"thread_id": "15"}},
  context=BankContext()
)


/Users/suvo/Documents/Langchain/lca-lc-foundations/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=BankContext(balance=900, ...wal=100, last_deposit=0), input_type=BankContext])
  return self.__pydantic_serializer__.to_python(
/Users/suvo/Documents/Langchain/lca-lc-foundations/.venv/lib/python3.12/site-packages/pydantic/main.py:464: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `none` - serialized value may not be as expected [field_name='context', input_value=BankContext(balance=900, ...wal=100, last_deposit=0), input_type=BankContext])
  return self.__pydantic_serializer__.to_python(


In [109]:
print(response["messages"][-1].content)

Your current balance is 1000. This reflects the deposit of 200 and the withdrawal of 100 you reported.
